In [5]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_2")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

In [8]:
df = spark.read.load('raw_sales.csv', format='csv', header=True, inferSchema=True)

In [9]:
for row in df.schema:
  print(row)

StructField('datesold', TimestampType(), True)
StructField('postcode', IntegerType(), True)
StructField('price', IntegerType(), True)
StructField('propertyType', StringType(), True)
StructField('bedrooms', IntegerType(), True)


# Самостоятельная работа к уроку 4
На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания.

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)
*  Стоимость последнего проданного дома до текущего


In [10]:
df.createOrReplaceTempView('df')

In [11]:
spark.sql('''
SELECT
  postcode,
  datesold,
  AVG(price) OVER (ORDER BY postcode, datesold ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING) AS AVG_PRICE_10_BEFORE,
  AVG(price) OVER (ORDER BY postcode, datesold ROWS BETWEEN 1 FOLLOWING AND 10 FOLLOWING) AS AVG_PRICE_10_AFTER,
  SUM(price) OVER (ORDER BY postcode, datesold ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING) AS PRICE_BEFORE
FROM df
ORDER BY postcode, datesold;''').show()

+--------+-------------------+-------------------+------------------+------------+
|postcode|           datesold|AVG_PRICE_10_BEFORE|AVG_PRICE_10_AFTER|PRICE_BEFORE|
+--------+-------------------+-------------------+------------------+------------+
|    2600|2007-07-08 00:00:00|               null|          708350.0|        null|
|    2600|2007-08-16 00:00:00|           327000.0|          698350.0|      327000|
|    2600|2007-12-05 00:00:00|           558500.0|          679350.0|      790000|
|    2600|2008-01-21 00:00:00|  647333.3333333334|          742850.0|      825000|
|    2600|2008-04-24 00:00:00|           564250.0|          786600.0|      315000|
|    2600|2008-05-30 00:00:00|           509900.0|          839200.0|      292500|
|    2600|2008-06-19 00:00:00|           479750.0|          868450.0|      329000|
|    2600|2008-07-29 00:00:00|           520500.0|          805750.0|      765000|
|    2600|2008-09-02 00:00:00|           571312.5|          715250.0|      927000|
|   

## Задание 2
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)) (ипользуйте udf). Попробуйте сделать то же самое используя pandas udf.

In [12]:
prices = spark.sql('''
SELECT
  postcode,
  datesold,
  AVG(price) OVER (ORDER BY YEAR(datesold)) AS AVG_PRICE_YEAR,
  AVG(price) OVER (ORDER BY postcode, datesold ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING) AS AVG_PRICE_10_BEFORE,
  AVG(price) OVER (ORDER BY postcode, datesold ROWS BETWEEN 1 FOLLOWING AND 10 FOLLOWING) AS AVG_PRICE_10_AFTER,
  SUM(price) OVER (ORDER BY postcode, datesold ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING) AS PRICE_BEFORE
FROM df
ORDER BY postcode, datesold;''')

# Я не совсем понимаю, что тут конкретно нужно сделать и как к этому прикрутить udf. 
# Буду очень признателен, если вы напишете в комментарии к ДЗ пример кода, который нужно было написать, спасибо!

# Задание 3
SQL like case when или if elif else

Создайте колонку, в которой в которой будет отображаться "+", "-" или "=", если "Средняя стомость 10 проданных домов до текущего в том же районе" больше, меньше или равно "Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)", соотвественно.

Если одно из полей Null, запишите в эту колонку "Нет данных"

In [ ]:
spark.sql('''
SELECT
  postcode,
  datesold,
  AVG(price) OVER (ORDER BY postcode, datesold ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING) AS AVG_PRICE_10_BEFORE,
  AVG(price) OVER (ORDER BY postcode, datesold ROWS BETWEEN 1 FOLLOWING AND 10 FOLLOWING) AS AVG_PRICE_10_AFTER,
  CASE
  WHEN AVG_PRICE_10_BEFORE IS NULL OR 
      AVG_PRICE_10_AFTER IS NULL THEN 'Нет данных'
  WHEN AVG_PRICE_10_BEFORE > AVG_PRICE_10_AFTER THEN "+"
  WHEN AVG_PRICE_10_BEFORE < AVG_PRICE_10_AFTER THEN "-"
  WHEN AVG_PRICE_10_BEFORE = AVG_PRICE_10_AFTER THEN "="
  END as price_comparison  
FROM df
ORDER BY postcode, datesold;''').show()

+--------+-------------------+-------------------+------------------+----------------+
|postcode|           datesold|AVG_PRICE_10_BEFORE|AVG_PRICE_10_AFTER|price_comparison|
+--------+-------------------+-------------------+------------------+----------------+
|    2600|2007-07-08 00:00:00|               null|          708350.0|      Нет данных|
|    2600|2007-08-16 00:00:00|           327000.0|          698350.0|               -|
|    2600|2007-12-05 00:00:00|           558500.0|          679350.0|               -|
|    2600|2008-01-21 00:00:00|  647333.3333333334|          742850.0|               -|
|    2600|2008-04-24 00:00:00|           564250.0|          786600.0|               -|
|    2600|2008-05-30 00:00:00|           509900.0|          839200.0|               -|
|    2600|2008-06-19 00:00:00|           479750.0|          868450.0|               -|
|    2600|2008-07-29 00:00:00|           520500.0|          805750.0|               -|
|    2600|2008-09-02 00:00:00|           57